## Data Fetching
This interactive notebook handles the fetching and cleaning of data from the EPA's Air Quality System. To do so, we should start by importing some of our required libraries, including our custom created `pyaqs` module that provides wrapper methods to convert information stored in the AQS REST API to easily accessible and modifiable Pandas dataframes.

In [69]:
from pyaqs import AQSFetcher
import pandas as pd

Now, we will instantiate a new AQSFetcher object and use it to get the required data from the EPA website. For now, we will focus our attention on counties within Illinois, the state where we currently reside. To do so, we will use some of the custom defined methods to get the appropriate identification codes for the necessary locations and parameters.

To note, in this context, a *parameter* is a compound that in the air that can be measured. The EPA has many such parameters, sorted into different classes whose descriptions are easily accessible through the API.

In [70]:
aqs_fetcher = AQSFetcher('bbjornstad.flatiron@gmail.com', 'ochrefox21')

In [71]:
state_codes = aqs_fetcher.get_state_codes()
state_codes.head()

,code,state_name
0,01,Alabama
1,02,Alaska
2,04,Arizona
3,05,Arkansas
4,06,California


Let's store the code for Illinois in a variable for easy access.

In [72]:
il_code = state_codes.loc[state_codes.state_name == 'Illinois', 'code'].values[0]
il_code

'17'

And now we will get a list of codes for the counties within Illinois.

In [73]:
il_county_codes = aqs_fetcher.get_counties_by_state(il_code)
il_county_codes.head()

,code,county_name
0,001,Adams
1,003,Alexander
2,005,Bond
3,007,Boone
4,009,Brown


Finally, let's take a look at the possible parameter classes and identify a set that seems reasonable for analysis.

In [74]:
aqs_fetcher.get_parameter_classes()

,class_name,class_description
0,AIRNOW MAPS,The parameters represented on AirNow maps (881...
1,ALL,Select all Parameters Available
2,AQI POLLUTANTS,Pollutants that have an AQI Defined
3,CORE_HAPS,Urban Air Toxic Pollutants
4,CRITERIA,Criteria Pollutants
5,CSN DART,List of CSN speciation parameters to populate ...
6,FORECAST,Parameters routinely extracted by AirNow (STI)
7,HAPS,Hazardous Air Pollutants
8,IMPROVE CARBON,IMPROVE Carbon Parameters
9,IMPROVE_SPECIATION,PM2.5 Speciated Parameters Measured at IMPROVE...


We are most interested in those parameters held in the CRITERIA class, as indicated by the description. In particular, this class defines pollutants that the EPA has determined to be suitable criteria for overall air quality.

In [75]:
parameter_codes = aqs_fetcher.get_parameter_list_by_class('CRITERIA')
parameter_codes

,code,parameter_description
0,14129,Lead (TSP) LC
1,42101,Carbon monoxide
2,42401,Sulfur dioxide
3,42602,Nitrogen dioxide (NO2)
4,44201,Ozone
5,81102,PM10 Total 0-10um STP
6,85129,Lead PM10 LC FRM/FEM
7,88101,PM2.5 - Local Conditions


Fantastic, these will allow us the possibility to easily partition and query the data that we need to continue with the analysis.

In [76]:
il_aq_data = aqs_fetcher.annual_data_by_state(il_code, parameter_codes.code, 20120101, 20161231)
il_aq_data.head()

,state_code,county_code,site_number,parameter_code,poc,latitude,longitude,datum,parameter,sample_duration,...,fiftieth_percentile,tenth_percentile,local_site_name,site_address,state,county,city,cbsa_code,cbsa,date_of_last_change
0,17,049,1001,44201,2,39.067159,-88.548934,WGS84,Ozone,8-HR RUN AVG BEGIN HOUR,...,0.042,0.029,CENTRAL JR HIGH,10421 N. US HWY. 45,Illinois,Effingham,None,20820,"Effingham, IL",2018-07-19
1,17,049,1001,44201,2,39.067159,-88.548934,WGS84,Ozone,8-HR RUN AVG BEGIN HOUR,...,0.042,0.029,CENTRAL JR HIGH,10421 N. US HWY. 45,Illinois,Effingham,None,20820,"Effingham, IL",2018-07-19
2,17,049,1001,44201,2,39.067159,-88.548934,WGS84,Ozone,8-HR RUN AVG BEGIN HOUR,...,0.042,0.029,CENTRAL JR HIGH,10421 N. US HWY. 45,Illinois,Effingham,None,20820,"Effingham, IL",2018-07-19
3,17,049,1001,44201,2,39.067159,-88.548934,WGS84,Ozone,1 HOUR,...,0.047,0.034,CENTRAL JR HIGH,10421 N. US HWY. 45,Illinois,Effingham,None,20820,"Effingham, IL",2018-07-19
4,17,049,1001,44201,2,39.067159,-88.548934,WGS84,Ozone,8-HR RUN AVG BEGIN HOUR,...,0.043,0.028,CENTRAL JR HIGH,10421 N. US HWY. 45,Illinois,Effingham,None,20820,"Effingham, IL",2018-07-19


In [77]:
il_aq_data.columns

Index(['state_code', 'county_code', 'site_number', 'parameter_code', 'poc',
       'latitude', 'longitude', 'datum', 'parameter', 'sample_duration',
       'pollutant_standard', 'metric_used', 'method', 'year',
       'units_of_measure', 'event_type', 'observation_count',
       'observation_percent', 'validity_indicator', 'valid_day_count',
       'required_day_count', 'exceptional_data_count',
       'null_observation_count', 'primary_exceedance_count',
       'secondary_exceedance_count', 'certification_indicator',
       'arithmetic_mean', 'standard_deviation', 'first_max_value',
       'first_max_datetime', 'second_max_value', 'second_max_datetime',
       'third_max_value', 'third_max_datetime', 'fourth_max_value',
       'fourth_max_datetime', 'first_max_nonoverlap_value',
       'first_max_n_o_datetime', 'second_max_nonoverlap_value',
       'second_max_n_o_datetime', 'ninety_ninth_percentile',
       'ninety_eighth_percentile', 'ninety_fifth_percentile',
       'ninetieth_perc

Let's also do some paring of this large number of columns. Many of these fields are superfluous for our analysis and so we can simply drop the columns (or in this case, keep the columns that we want).

In [78]:
cols_to_drop = ['state_code', 'poc', 'latitude', 'longitude', 'datum', 'event_type', 'observation_percent', 'validity_indicator',
                'valid_day_count', 'required_day_count', 'primary_exceedance_count', 'secondary_exceedance_count', 
                'certification_indicator', 'first_max_value', 'first_max_datetime', 'second_max_value', 'second_max_datetime',
                'third_max_value', 'third_max_datetime', 'fourth_max_value', 'fourth_max_datetime', 'first_max_nonoverlap_value',
                'first_max_n_o_datetime', 'second_max_nonoverlap_value', 'second_max_n_o_datetime', 'ninety_ninth_percentile',
                'ninety_eighth_percentile', 'ninety_fifth_percentile', 'ninetieth_percentile', 'seventy_fifth_percentile',
                'fiftieth_percentile', 'tenth_percentile', 'cbsa_code', 'cbsa', 'pollutant_standard', 'method', 'metric_used']
il_aq_data.drop(columns=cols_to_drop, inplace=True)

In [79]:
il_aq_data.head()

,county_code,site_number,parameter_code,parameter,sample_duration,year,units_of_measure,observation_count,exceptional_data_count,null_observation_count,arithmetic_mean,standard_deviation,local_site_name,site_address,state,county,city,date_of_last_change
0,049,1001,44201,Ozone,8-HR RUN AVG BEGIN HOUR,2016,Parts per million,3633,0,0,0.042493,0.010552,CENTRAL JR HIGH,10421 N. US HWY. 45,Illinois,Effingham,None,2018-07-19
1,049,1001,44201,Ozone,8-HR RUN AVG BEGIN HOUR,2016,Parts per million,5127,0,0,0.042453,0.010575,CENTRAL JR HIGH,10421 N. US HWY. 45,Illinois,Effingham,None,2018-07-19
2,049,1001,44201,Ozone,8-HR RUN AVG BEGIN HOUR,2016,Parts per million,5127,0,0,0.042453,0.010575,CENTRAL JR HIGH,10421 N. US HWY. 45,Illinois,Effingham,None,2018-07-19
3,049,1001,44201,Ozone,1 HOUR,2016,Parts per million,5120,0,16,0.047056,0.010837,CENTRAL JR HIGH,10421 N. US HWY. 45,Illinois,Effingham,None,2018-07-19
4,049,1001,44201,Ozone,8-HR RUN AVG BEGIN HOUR,2015,Parts per million,3447,0,0,0.042040,0.010396,CENTRAL JR HIGH,10421 N. US HWY. 45,Illinois,Effingham,None,2018-07-19


Let's also do a bit of investigation into the consistency of the data. In particular, there are various possible values represented in the `units_of_measure` field, leading us to suspect that perhaps we will need to do a bit of unit conversion before we are ready to begin analysis. To check if this is the case, we can group by the parameter name toogether with the units of measure. If we see that each name is associated to only a single unit, then we will know that we won't have to perform any unit conversions in order to compare within each compound present in the air.

In [80]:
il_aq_data.groupby(['parameter', 'units_of_measure']).count()

,,county_code,site_number,parameter_code,sample_duration,year,observation_count,exceptional_data_count,null_observation_count,arithmetic_mean,standard_deviation,local_site_name,site_address,state,county,city,date_of_last_change
parameter,units_of_measure,,,,,,,,,,,,,,,,
Carbon monoxide,Parts per million,46,46,46,46,46,46,46,46,46,46,46,46,46,46,34,46
Lead (TSP) LC,Micrograms/cubic meter (LC),62,62,62,62,62,62,62,62,62,62,62,62,62,62,58,62
Nitrogen dioxide (NO2),Parts per billion,68,68,68,68,68,68,68,68,68,68,68,68,68,68,68,68
Ozone,Parts per million,748,748,748,748,748,748,748,748,748,748,748,748,748,748,608,748
PM10 Total 0-10um STP,Micrograms/cubic meter (25 C),35,35,35,35,35,35,35,35,35,35,35,35,35,35,35,35
PM2.5 - Local Conditions,Micrograms/cubic meter (LC),701,701,701,701,701,701,701,701,701,701,701,701,701,701,633,701
Sulfur dioxide,Parts per billion,311,311,311,311,311,311,311,311,311,311,307,311,311,311,269,311


We see that in fact, it is the case that the units have already been standardized. Therefore, we don't need to do any unit conversion if we want to gain some actionable insights when making comparisons within a particular parameter. We also see that we have a lot more readings for the Ozone and PM 2.5 parameters.

In [91]:
il_county_means = il_aq_data.groupby(['county', 'parameter', 'units_of_measure']).mean()
il_county_means

year  \
county    parameter                units_of_measure                           
Adams     Ozone                    Parts per million            2014.000000   
Champaign Carbon monoxide          Parts per million            2014.250000   
          Ozone                    Parts per million            2014.000000   
          PM2.5 - Local Conditions Micrograms/cubic meter (LC)  2015.147059   
          Sulfur dioxide           Parts per billion            2014.136364   
...                                                                     ...   
Will      PM2.5 - Local Conditions Micrograms/cubic meter (LC)  2015.172414   
Winnebago Carbon monoxide          Parts per million            2012.000000   
          Lead (TSP) LC            Micrograms/cubic meter (LC)  2012.500000   
          Ozone                    Parts per million            2014.000000   
          PM2.5 - Local Conditions Micrograms/cubic meter (LC)  2015.000000   

                                                                observation_count  \
county    parameter                units_of_measure                                 
Adams     Ozone                    Parts per million                  5809.400000   
Champaign Carbon monoxide          Parts per million                 21264.000000   
          Ozone                    Parts per million                  7721.025000   
          PM2.5 - Local Conditions Micrograms/cubic meter (LC)         571.205882   
          Sulfur dioxide           Parts per billion                 13005.545455   
...                                                                           ...   
Will      PM2.5 - Local Conditions Micrograms/cubic meter (LC)         122.551724   
Winnebago Carbon monoxide          Parts per million                  8664.500000   
          Lead (TSP) LC            Micrograms/cubic meter (LC)          55.500000   
          Ozone                    Parts per million                  7905.850000   
          PM2.5 - Local Conditions Micrograms/cubic meter (LC)          80.333333   

                                                                exceptional_data_count  \
county    parameter                units_of_measure                                      
Adams     Ozone                    Parts per million                               0.0   
Champaign Carbon monoxide          Parts per million                               0.0   
          Ozone                    Parts per million                               0.0   
          PM2.5 - Local Conditions Micrograms/cubic meter (LC)                     0.0   
          Sulfur dioxide           Parts per billion                               0.0   
...                                                                                ...   
Will      PM2.5 - Local Conditions Micrograms/cubic meter (LC)                     0.0   
Winnebago Carbon monoxide          Parts per million                               0.0   
          Lead (TSP) LC            Micrograms/cubic meter (LC)                     0.0   
          Ozone                    Parts per million                               0.0   
          PM2.5 - Local Conditions Micrograms/cubic meter (LC)                     0.0   

                                                                null_observation_count  \
county    parameter                units_of_measure                                      
Adams     Ozone                    Parts per million                         79.400000   
Champaign Carbon monoxide          Parts per million                       1450.250000   
          Ozone                    Parts per million                        128.325000   
          PM2.5 - Local Conditions Micrograms/cubic meter (LC)               97.617647   
          Sulfur dioxide           Parts per billion                        543.818182   
...                                                                                ...   
Will      PM2.5 - Local Conditions Micrograms/

In [92]:
il_county_counts = il_aq_data.groupby(['county', 'parameter', 'units_of_measure']).count()
il_county_counts

county_code  \
county    parameter                units_of_measure                           
Adams     Ozone                    Parts per million                     20   
Champaign Carbon monoxide          Parts per million                     12   
          Ozone                    Parts per million                     40   
          PM2.5 - Local Conditions Micrograms/cubic meter (LC)           34   
          Sulfur dioxide           Parts per billion                     22   
...                                                                     ...   
Will      PM2.5 - Local Conditions Micrograms/cubic meter (LC)           29   
Winnebago Carbon monoxide          Parts per million                      2   
          Lead (TSP) LC            Micrograms/cubic meter (LC)            2   
          Ozone                    Parts per million                     20   
          PM2.5 - Local Conditions Micrograms/cubic meter (LC)           12   

                                                                site_number  \
county    parameter                units_of_measure                           
Adams     Ozone                    Parts per million                     20   
Champaign Carbon monoxide          Parts per million                     12   
          Ozone                    Parts per million                     40   
          PM2.5 - Local Conditions Micrograms/cubic meter (LC)           34   
          Sulfur dioxide           Parts per billion                     22   
...                                                                     ...   
Will      PM2.5 - Local Conditions Micrograms/cubic meter (LC)           29   
Winnebago Carbon monoxide          Parts per million                      2   
          Lead (TSP) LC            Micrograms/cubic meter (LC)            2   
          Ozone                    Parts per million                     20   
          PM2.5 - Local Conditions Micrograms/cubic meter (LC)           12   

                                                                parameter_code  \
county    parameter                units_of_measure                              
Adams     Ozone                    Parts per million                        20   
Champaign Carbon monoxide          Parts per million                        12   
          Ozone                    Parts per million                        40   
          PM2.5 - Local Conditions Micrograms/cubic meter (LC)              34   
          Sulfur dioxide           Parts per billion                        22   
...                                                                        ...   
Will      PM2.5 - Local Conditions Micrograms/cubic meter (LC)              29   
Winnebago Carbon monoxide          Parts per million                         2   
          Lead (TSP) LC            Micrograms/cubic meter (LC)               2   
          Ozone                    Parts per million                        20   
          PM2.5 - Local Conditions Micrograms/cubic meter (LC)              12   

                                                                sample_duration  \
county    parameter                units_of_measure                               
Adams     Ozone                    Parts per million                         20   
Champaign Carbon monoxide          Parts per million                         12   
          Ozone                    Parts per million                         40   
          PM2.5 - Local Conditions Micrograms/cubic meter (LC)               34   
          Sulfur dioxide           Parts per billion                         22   
...                                                                         ...   
Will      PM2.5 - Local Conditions Micrograms/cubic meter (LC)               29   
Winnebago Carbon monoxide          Parts per million                          2   
          Lead (TSP) LC            Micrograms/cubic meter (LC)                2   
          Ozone             

In [88]:
county_cancer = pd.read_csv('cleaned_data/il_all_counties_cancer_rates.csv')
county_cancer.drop(columns=['Unnamed: 0'], inplace=True)

In [89]:
county_cancer

,County,Year,AgeAdjustedRate,CaseCount,Population
0,Kane,2012-2016,51.2,1298,2633345
1,DuPage,2012-2016,51.8,2643,4657275
2,Champaign,2012-2016,52.8,499,1034430
3,Woodford,2012-2016,54.8,135,194822
4,Stephenson,2012-2016,52.4,186,231025
...,...,...,...,...,...
97,Crawford,2012-2016,73.2,100,96691
98,Clark,2012-2016,77.6,88,80111
99,Macon,2012-2016,93.4,701,542237
100,Iroquois,2012-2016,78.2,171,143671
